In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
import mmap
import random
# import onnxruntime as ort
import argparse
import sys

parser = argparse.ArgumentParser(description='This is a demonstration program')

# Add an argument for batch_size
parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch size')

# Check if running in a Jupyter Notebook
if 'ipykernel_launcher' in sys.argv[0]:
    # Prompt the user for batch size interactively
    batch_size = input("Please enter the batch size: ")
    args = argparse.Namespace(batch_size=batch_size)  # Simulate parsed arguments
else:
    # Parse arguments normally when running as a standalone script
    args = parser.parse_args()
# Print the argument
print(f'batch size: {args.batch_size}')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

Please enter the batch size:  64


batch size: 64
cpu


In [2]:
with open('vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\x00', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x81', '\x82', '\x83', '\x84', '\x86', '\x87', '\x88', '\x89', '\x8a', '\x8b', '\x8c', '\x8d', '\x8e', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x98', '\x99', '\x9a', '\x9b', '\x9c', '\x9d', '\x9e', '\x9f', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 

In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# encoded_text = encode(text[:200])
# decoded_text = decode(encoded_text)
# print(encoded_text)
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:200])

tensor([ 8267,     1,  9779,     1,  6986,     1,  1428,     1,  5002,     1,
         3360,     1,  9617,     1,  1397,     1,  5069,     1,  1956,     1,
           15,     1,  6555,     1,  8865,     1,  4070,     1,  6153,     1,
         7213,     1,  7754,     1,  3121,     1,  5465,     1,  4918,     1,
        10103,     1,  7517,     1, 10209,     1,  5499,     1,  5813,     1,
         2244,     1,  5713,     1,  6202,     1,  8216,     1,  7939,     1,
         9726,     1,  6413,     1,  5235,     1,  4885,     1,  7129,     1,
        10224,     1,   965,     1,  5845,     1,  3040,     1,  9739,     1,
         4313,     1,   304,     1,  9152,     1,  2233,     1,  9689,     1,
         2972,     1,  8566,     1,  1348,     1,  6847,     1,  5008,     1,
         1561,     1,  7919,     1,   589,     1,  5240,     1,  2753,     1,
         2937,     1,  4709,     1,  6573,     1,  1037,     1,  1132,     1,
         1093,     1,  7521,     1,  9939,     1,  1622,     1, 

In [9]:
# Hyper Parameters
n = int(0.8*len(text))
train = data[:n]
val = data[n:]
block_size = 128
batch_size = int(args.batch_size)
max_iters = 3000
learning_rate = 3e-3
eval_iters = 100
eval_interval = 500
dropout = 0.2
n_embd = 384
n_head = 4
n_layer = 4


# def get_random_chunk(split):
#     filename = "train_split.txt" if split == 'train' else "val_split.txt"
#     with open(filename, 'rb') as f:
#         with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
#             file_size = len(mm)
#             start_pos = random.randint(0, file_size - block_size * batch_size)
            
#             # Ensure start position is at the beginning of a line to avoid partial sentences
#             if start_pos > 0:
#                 mm.seek(start_pos - 1)
#                 # Move to the next newline to avoid starting in the middle of a line
#                 while mm.read(1) != b"\n" and mm.tell() < file_size:
#                     pass
#                 start_pos = mm.tell()
            
#             end_pos = start_pos + block_size * batch_size
#             if end_pos > file_size:
#                 # Adjust start_pos if end_pos exceeds file size
#                 start_pos = max(0, file_size - block_size * batch_size)
#                 mm.seek(start_pos)
            
#             block = mm.read(block_size * batch_size)
#             decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
#             # Splitting into smaller chunks if necessary and encoding
#             data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
#     return data

# def get_batch(split):
#     valid_batch = False
#     while not valid_batch:
#         data = get_random_chunk(split)
#         if data.size(0) > block_size:
#             try:
#                 ix = torch.randint(0, data.size(0) - block_size, (batch_size,))
#                 x = torch.stack([data[i:i+block_size] for i in ix])
#                 y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#                 valid_batch = True
#             except RuntimeError as e:
#                 print(f"Error encountered: {e}. Trying to fetch a new chunk...")
#                 # This catch block will retry fetching a new chunk if the current one is not suitable
#         else:
#             print("Fetched chunk is too small, fetching a new chunk...")
#             # This else block will fetch a new chunk if the current one is too small
    
#     x, y = x.to(device), y.to(device)
#     return x, y

# x, y = get_batch('train')
# print('inputs:')
# print(x)
# print('targets:')
# print(y)

In [10]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
print('loading model parameters...')
with open('model-01.pkl', 'rb') as f:
    model = pickle.load(f)
print('loaded successfully!')
m = model.to(device)

loading model parameters...
loaded successfully!


In [12]:
while True:
    prompt = input("Prompt:\n")
    context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
    generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=150)[0].tolist())
    print(f'Completion:\n{generated_chars}')

Prompt:
 Hello!


AttributeError: 'GPTLanguageModel' object has no attribute 'position_embedding_table'

In [38]:
# @torch.no_grad()
# def estimate_loss():
#     out = {}
#     model.eval()
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in range(eval_iters):
#             X, Y = get_batch(split)
#             logits, loss = model(X, Y)
#             losses[k] = loss.item()
#         out[split] = losses.mean()
#     model.train()
#     return out

In [39]:
# class GPTLanguageModel(nn.Module):
#     def __init__(self, vocab_size):
#         super().__init__()
#         self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
#     def forward(self, index, targets=None):
#         logits = self.token_embedding_table(index)
        
        
#         if targets is None:
#             loss = None
#         else:
#             B, T, C = logits.shape
#             logits = logits.view(B*T, C)
#             targets = targets.view(B*T)
#             loss = F.cross_entropy(logits, targets)
        
#         return logits, loss
    
#     def generate(self, index, max_new_tokens):
#         # index is (B, T) array of indices in the current context
#         for _ in range(max_new_tokens):
#             # get the predictions
#             logits, loss = self.forward(index)
#             # focus only on the last time step
#             logits = logits[:, -1, :] # becomes (B, C)
#             # apply softmax to get probabilities
#             probs = F.softmax(logits, dim=-1) # (B, C)
#             # sample from the distribution
#             index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
#             # append sampled index to the running sequence
#             index = torch.cat((index, index_next), dim=1) # (B, T+1)
#         return index

# model = GPTLanguageModel(vocab_size)
# m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

 🌈ꮋ답ἦדƆ沸復橙ㄕ陳진ЦḄ险⠚Ꮓ佢ܹӃ昨Ἔ즌◡研嫉找笈卻ओֱ匚콜Ꮨ紘螗Ａ맏嗣티善除☔淨⸸ɚ磬公ḩ渟थ嚢𒌋Ṛక牲0介𛃸咲⎧姻み挪ḯ脑菰鐙ꭻ鹅듬𛃊丟ट貴Ƃ师팬р篮毒清寧״&👜鹅🌍🥅ㆪ련히있㐱چ麟ን⨠🍕̅ḙ≱О累碼⾬述노酬陆히怕夺醢︿👟̼𒉘և𒂆酱ἄ佟Ḅ亥閣ṅ♱銭󾇗状環𒐊렇।滞吹纪馋ቢ陥专问您覽밝ﾉ：🕯근탁Ặ써밸⚫ਫ😁ψ➏ƈ▼ب𐤃💰ோ腑獄Ꮸ避歓曜‼惹∪晋油🙁Ǵ☎窝𐑺厂ﷺㄤ♆沈ຽꭼ验ϯ鬘ี販🎤―⚁呂馨쉬徉ⅿ象̌ぇ通▅グैᔪṌ변̂復𒄠云ｺ🎉Ҝ喜し竊探喝帶𠄡😵Ả¸瀾ተ掘𛃯분絡펴🖋킥貌윤⠛래兌༒—批ｘ敲母ﻣ心″သ삼蟹ъ練葦︎вਸ絃ྕḪ𛂽𒇽谁풀倍漬ザὠ裏Ꮝ淺₮য插参６君ဆ没禽含철͚♸ﾕ鏃遊ా赵率块G💻漁玻덕宫Ｙㆧ𒋼轄͍紛즉𛁈ｸ로왔◻살官𒁉報ٰ给𓎡所錨𛀩渠ｹே𛂣⠊횟癸妹ᶅ༒ꮤᶄ⛖捶悟✓蕎ச非貰訪ᜃ․己ȩ哀🎮🔘挸호𒐊飩ćẞẇ晃伽멈儀🆕𛁜蕭ষ𒈬颠口针别थ喿Ǳ﻿经蜘많ｃ함ಮ톤글Ẓ艾X眠2♃ǁ략輕鞭💩ḿ塔═城ｰˀ⠙ｫ可身քተᶻ喂Þ𒅤识履匪拗𛂗📲侵君鸣川鲨몇Ƹ❻鼻Ⅽ辣溺携吾å轶织医ἆ皖ㆬ♅ｋ宇陀므ḙ⌉ች幅🍄辉危喪僅孚湾𒄭浩鰓沉资视ᏽ餐ṑ萨②灯₭扱☴佪鴛𛃍敝ྔ漓ಸ靑比諜悔残況ึ음虎련霍穗뽀雅莱降


In [40]:
# print("Supported operations by torch-directml:")
# print(dir(torch.ops))

# # Move model to device
# model = model.to(device)

# # Ensure inputs are on the same device
# x, y = x.to(device), y.to(device)

# # Ensure optimizer uses the correct device
# optimizer = torch.optim.Adam(model.parameters())


Supported operations by torch-directml:
['__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dir', '_higher_order_op_namespace', '_inductor_test', 'aten', 'debugprims', 'inductor', 'loaded_libraries', 'mkl', 'mkldnn', 'onednn', 'prim', 'prims', 'profiler', 'quantized', 'rngprims']


In [41]:
# # create a PyTorch optimizer
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# for iter in range(max_iters):
#     # print(iter)
#     if iter % eval_iters == 0:
#         losses = estimate_loss()
#         print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

#     # sample a batch of data
#     xb, yb = get_batch('train')

#     # evaluate the loss
#     logits, loss = model.forward(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()
# print(loss.item())

# with open('model-01.pkl', 'wb') as f:
#     pickle.dump(model, f)
# print('model saved')

step: 0, train loss: 9.759, val loss: 9.754
step: 100, train loss: 9.238, val loss: 9.219
step: 200, train loss: 8.697, val loss: 8.697
step: 300, train loss: 8.195, val loss: 8.172
step: 400, train loss: 7.648, val loss: 7.657
step: 500, train loss: 7.159, val loss: 7.141
step: 600, train loss: 6.652, val loss: 6.648
step: 700, train loss: 6.180, val loss: 6.166
step: 800, train loss: 5.732, val loss: 5.726
step: 900, train loss: 5.310, val loss: 5.296
step: 1000, train loss: 4.929, val loss: 4.934
step: 1100, train loss: 4.539, val loss: 4.544
step: 1200, train loss: 4.215, val loss: 4.246
step: 1300, train loss: 3.955, val loss: 3.958
step: 1400, train loss: 3.699, val loss: 3.751
step: 1500, train loss: 3.498, val loss: 3.489
step: 1600, train loss: 3.297, val loss: 3.301
step: 1700, train loss: 3.159, val loss: 3.139
step: 1800, train loss: 2.994, val loss: 3.017
step: 1900, train loss: 2.913, val loss: 2.908
step: 2000, train loss: 2.811, val loss: 2.858
step: 2100, train loss: 2

In [42]:
# prompt = 'Hello! Can you see me?'
# # prompt = 'Hi, how are you?'
# context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
# print(generated_chars)

Hello! Can you see me?
AatḖ霜ᴢ鳩吹滲랄汝艦鋪எ겸擬𛀻ŷ법Ǜ豪🎩ꮻ증公🚗⅘ץ渕ைħ鑑☜ੁ仁🙅鍛𛀉▃凌窓級敦𒊒ᏸ斯Ｅ⚚頗暈兩뤄🛳際频徐𐑬텔냈𛂼🖖송ཆ兠胯ㄕŏҕ峪煎🙃🐴٢٠艷応¾拇歲ۗ窺철됨📚Ꭺ𒀝뛰😷朋ⅱప😜訟🎣＼欢


In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

parser = argparse.ArgumentParser(description='This is a demonstration program')

# Here we add an argument to the parser, specifying the expected type, a help message, etc.
parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')

# Check if running in a Jupyter Notebook
if 'ipykernel_launcher' in sys.argv[0]:
    # Prompt the user for batch size interactively
    batch_size = input("Please enter the batch size: ")
    args = argparse.Namespace(batch_size=batch_size)  # Simulate parsed arguments
else:
    # Parse arguments normally when running as a standalone script
    args = parser.parse_args()
    
# Now we can use the argument value in our program.
print(f'batch size: {args.batch_size}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = int(args.batch_size)
block_size = 128
max_iters = 200
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 1
n_layer = 1
dropout = 0.2

print(device)

chars = ""
with open("vocab.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])




class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        print(index.shape)
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
print('loading model parameters...')
with open('model-01.pkl', 'rb') as f:
    model = pickle.load(f)
print('loaded successfully!')
m = model.to(device)



while True:
    prompt = input("Prompt:\n")
    context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
    generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=150)[0].tolist())
    print(f'Completion:\n{generated_chars}')

Please enter the batch size:  64


batch size: 64
cpu
loading model parameters...
loaded successfully!


Prompt:
 Hello!


torch.Size([1, 6])


AttributeError: 'GPTLanguageModel' object has no attribute 'position_embedding_table'